# Analyzing DBA Salaries

In this demo, we will look at a survey of Database Administrator salaries that Brent Ozar put together in 2017.  Our goal is to see if we can find important features which explain salaries and help predict how much we might make according to this data sample.

This data is available on the [Brent Ozar Unlimited](https://www.brentozar.com/archive/2017/01/2017-data-professional-salary-survey-results/) website.

Because this is an Excel file, we will use the `XLConnect` package to read the data set.  We will build a regression tree using the `evtree` package--note that this is different from `rgp`, which we have used so far for genetic algorithms.  `evtree` makes it easier to build decision trees and regression trees, so we will take advantage of this relative ease of use.

In [ ]:
if(!require(evtree)) {
    install.packages("evtree", repos = "http://cran.us.r-project.org")
    library(evtree)
}

if(!require(XLConnect)) {
  install.packages("XLConnect", repos = "http://cran.us.r-project.org")
  library(XLConnect)
}

if(!require(tidyverse)) {
  install.packages("tidyverse", repos = "http://cran.us.r-project.org")
  library(tidyverse)
}

In [ ]:
wb <- loadWorkbook("Data/2017_Data_Professional_Salary_Survey_Responses.xlsx")
salary_data <- readWorksheet(wb, sheet = "Salary Survey", region = "A4:T2902")

Let's take a quick look at the data to make sure everything loaded correctly and also to give us an idea of what we're dealing with here.

In [ ]:
str(salary_data)

In [ ]:
head(salary_data)

For our regression, we are going to perform some data munging, creating a data frame which fits our guidelines.  These guidelines are:
- We will only look at participants in the United States.  There are some huge differences in average salary across countries that I don't want to deal with today.
- We will look only at participants who selected Microsoft SQL Server as their primary database.  This is a vast majority of the respondants, so that won't filter out much.
- We will look only at full-time employees, skipping freelancers and part-timers.
- We will only look at people with a salary below $300,000 per year.  Values above that are possible, but they're likely to be typos or bad data.

From there, we want to pull out a few variables and include them as-is:
* SalaryUSD (our independent variable)
* YearsWithThisDatabase
* HoursWorkedPerWeek

We would also like to convert some variables to factors and include them as well:
* Education
* EmploymentSector
* ManageStaff

In [ ]:
salary_pred <- salary_data %>%
  filter(Country == "United States") %>%
  filter(PrimaryDatabase == "Microsoft SQL Server") %>%
  filter(EmploymentStatus == "Full time employee") %>%
  filter(SalaryUSD <= 300000) %>%
  select(SalaryUSD, YearsWithThisDatabase, HoursWorkedPerWeek, Education, EmploymentSector, ManageStaff)

In [ ]:
salary_pred$Education <- as.factor(salary_pred$Education)
salary_pred$EmploymentSector <- as.factor(salary_pred$EmploymentSector)
salary_pred$ManageStaff <- as.factor(salary_pred$ManageStaff)

We have our data in place, so let's take a quick look at it to make sure everything looks fine.

In [ ]:
str(salary_pred)

In [ ]:
head(salary_pred, 6)

Our next goal is to build a regression tree.  Regression trees are sort of like decision trees, except instead of making yes-no decisions, we get expected outputs.  Our goal is to predict SalaryUSD given everything else in `salary_pred`.  We will set the maximum depth to 4, meaning that no branch may go further than four nodes deep.

In [ ]:
ev <- evtree(SalaryUSD ~ YearsWithThisDatabase + HoursWorkedPerWeek + Education + EmploymentSector + ManageStaff,
             data = salary_pred, minbucket = 10, maxdepth = 4)

After spending some time building up the tree, we can use the `plot` function to plot the results.

In [ ]:
plot(ev)

That's a somewhat noisy image, so let's look at the object details to see some more of this.

In [ ]:
ev

This gives us some insight into our variables.  First of all, hours worked per week was not a significant factor in this analysis.  Years with this database, though, is a critical factor--it shows up in a couple places.  Also interesting is that federal government and private business have nicer returns than state or local government.

We can use the `predict` function to see the specific predictions for each participant, and add those predictions as a new column called `EVPrediction`.

In [ ]:
salary_pred$EVPrediction <- predict(ev)

I'd like to compare this against a linear regression to see how well our regression tree stands up against the normal process.

In [ ]:
salary_lm <- lm(SalaryUSD ~ YearsWithThisDatabase + HoursWorkedPerWeek + Education + EmploymentSector + ManageStaff,
                data = salary_pred)

In [ ]:
summary(salary_lm)

These results are a bit different than the regression tree, so it'll be interesting to see how the aggregates shake out.  Let's predict each salary using this model and move forward.

In [ ]:
salary_pred$LMPrediction <- predict(salary_lm)

I'd like to use Root Mean Square Deviance to calculate the distance of both of these regressions from the correct answers.  The output of this RMSE calculation is dollars, so it gives us an idea of just how much variance we haven't explained.  Let's start with the regression tree and follow up with our linear regression.  Lower numbers are better.

In [ ]:
sqrt(mean((salary_pred$SalaryUSD - salary_pred$EVPrediction)^2))

In [ ]:
sqrt(mean((salary_pred$SalaryUSD - salary_pred$LMPrediction)^2))

Both of these ended up in a pretty similar place, with the linear model being just a hair more accurate.

## Part 2:  Individual Predictions

Now that we have a pair of models in place, let's generate predictions for three people.  The first person is someone brand new to the industry, the second person is me five years into my career, and the third is me today.  We can see how the two solutions fare when looking at these three individuals.

In [ ]:
test_cases <- data.frame(
    YearsWithThisDatabase = c(0, 5, 11),
    HoursWorkedPerWeek = c(40, 40, 40),
    Education = c("Bachelors (4 years)", "Masters", "Masters"),
    EmploymentSector = c("Private business", "State/province government", "Private business"),
    ManageStaff = c("No", "No", "Yes")
)

In [ ]:
test_cases

In [ ]:
predict(ev, test_cases)

In [ ]:
predict(salary_lm, test_cases)

In my test examples, the first and third cases end up being pretty close (within a couple thousand dollars), but the regression tree disagrees strongly with the linear model for a state employee with a Master's degree and 5 years of experience.